In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import pandas as pd

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Image preprocessing
data_transform = transforms.Compose([
    transforms.Resize((288, 288)),
    transforms.ToTensor()
])

# Load training and test datasets
train_dataset = datasets.ImageFolder('./data/plant_pathology/train', transform=data_transform)
test_dataset = datasets.ImageFolder('./data/plant_pathology/test', transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Classification and evaluation function
def train_and_evaluate(model_name="resnet18"):
    print(f"\n📌 Training model: {model_name}")
    
    if model_name == "resnet18":
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 2)
    elif model_name == "vgg16":
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)
    else:
        raise ValueError("Model name must be 'resnet18' or 'vgg16'")
    
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    
    # Training
    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/10 - Loss: {running_loss:.4f}")
    
    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total * 100
    print(f"✅ {model_name} Accuracy: {accuracy:.2f}%")
    return accuracy

# Run both models
resnet_acc = train_and_evaluate("resnet18")
vgg_acc = train_and_evaluate("vgg16")

# Save results to CSV
results_df = pd.DataFrame([
    {"Model": "ResNet-18", "Accuracy": round(resnet_acc, 2)},
    {"Model": "VGG16", "Accuracy": round(vgg_acc, 2)}
])

csv_path = "classification_results.csv"
results_df.to_csv(csv_path, index=False)

# Print results
print("\n📊 Final Classification Accuracy")
print(results_df.to_string(index=False))
print(f"\n📁 Results saved to: {csv_path}")

c:\Users\Grace\anaconda3\envs\singan\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



📌 Training model: resnet18
Epoch 1/10 - Loss: 1.9240
Epoch 2/10 - Loss: 0.0269
Epoch 3/10 - Loss: 0.0406
Epoch 4/10 - Loss: 0.0094
Epoch 5/10 - Loss: 0.0061
Epoch 6/10 - Loss: 0.0055
Epoch 7/10 - Loss: 0.0051
Epoch 8/10 - Loss: 0.0020
Epoch 9/10 - Loss: 1.1935
Epoch 10/10 - Loss: 0.1040
✅ resnet18 Accuracy: 53.85%

📌 Training model: vgg16
Epoch 1/10 - Loss: 1.8252
Epoch 2/10 - Loss: 0.0002
Epoch 3/10 - Loss: 0.0000
Epoch 4/10 - Loss: 0.0000
Epoch 5/10 - Loss: 0.0000
Epoch 6/10 - Loss: 0.0000
Epoch 7/10 - Loss: 0.0000
Epoch 8/10 - Loss: 0.0000
Epoch 9/10 - Loss: 0.0000
Epoch 10/10 - Loss: 0.0000
✅ vgg16 Accuracy: 53.85%

📊 Final Classification Accuracy
    Model  Accuracy
ResNet-18     53.85
    VGG16     53.85

📁 Results saved to: classification_results.csv
